In [ ]:
from model import Mamba, ModelArgs
from transformers import AutoTokenizer

In [3]:

# One of:
#     'state-spaces/mamba-2.8b-slimpj'
#     'state-spaces/mamba-2.8b'
#     'state-spaces/mamba-1.4b'
#     'state-spaces/mamba-790m'
#     'state-spaces/mamba-370m'
#     'state-spaces/mamba-130m'
pretrained_model_name = 'state-spaces/mamba-370m'

model = Mamba.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [10]:
import torch
import torch.nn.functional as F


def generate(model,
             tokenizer,
             prompt: str,
             n_tokens_to_gen: int = 50,
             sample: bool = True,
             top_k: int = 40):
    model.eval()

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')

    for token_n in range(n_tokens_to_gen):
        with torch.no_grad():
            indices_to_input = input_ids
            next_token_logits = model(indices_to_input)[:, -1]

        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape

        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)

        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]

        input_ids = torch.cat([input_ids, next_indices], dim=1)

    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

    return output_completions

def stream_generation(model,
                      tokenizer,
                      prompt: str,
                      n_tokens_to_gen: int = 512,
                      sample: bool = True,
                      top_k: int = 40,
                      device='cuda'
                      ):
    model.eval()
    done = False
    new_tokens = 0
    last_inputs = prompt
    last_ids = tokenizer(last_inputs, return_tensors='pt').input_ids.to('device')
    stop_token = tokenizer.eos_token_id
    next_indices=None
    while next_indices==stop_token or new_tokens < n_tokens_to_gen:

        indices_to_input = last_ids
        next_token_logits = model(indices_to_input)[:, -1]
        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape
        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)

        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]
        last_ids = torch.cat([last_ids, next_indices],dim=1)
        new_text = tokenizer.decode(next_indices[0])
        new_tokens += 1
        yield new_text

In [3]:
model.to('cuda')

NameError: name 'model' is not defined

In [65]:
print('def quick_sort(',end='')
for text in stream_generation(model, tokenizer, 'def quick_sort('):
  print(text, end='')

John: Hi!
Sally: Hi, John!
John: Hey there :)
Sally: Hey, I saw this movie with the same title but no sound.
John: Yes, you did!
Sally: I had a bad experience with the cast at the premiere!
John: I was so scared!
Sally: What a scene you did, and I was worried from the start!
John: Noooo, no!
Sally: What if I'd done it as a girl?
John: I'd tell everyone how brilliant you were.
Sally: My brother told him I had a bad voice.
John: He doesn't believe you!
Sally: The only time he listened was when his mother cried.
John: What a guy!
Sally: That's because you did it like an angel!
John: You're my inspiration for all future projects.
Sally: I'm going to make a movie that's better than you,
John: I've been working on it for years.
Sally: It's like watching yourself work,
John: that's why I want you to do this movie.
Sally: Really?
John: Yes.
Sally: Oh, thanks!
John: I'm so inspired.
Sally: Can I help?
John: Really!
Sally: Oh, no, no!
John: You didn't.
Sally: Heh, I thought you could come help m

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 1.06 MiB is free. Process 4235 has 14.74 GiB memory in use. Of the allocated memory 14.32 GiB is allocated by PyTorch, and 309.82 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [11]:
print(generate(model, tokenizer, 'John: Hi!\nSally:'))

John: Hi!
Sally: Oh
Sally: No
John: Yeah
John: What is your favourite color?
Sally: What?
Sally: Why don't you paint your own hair? (laughes)
John: That is so funny


In [17]:
print(generate(model, tokenizer, 'def reverse_string(',n_tokens_to_gen=512))

def reverse_string(self,string,reverse):
    n = 0
    reverse_start = reverse_start % string.length
    reverse_end = reverse_end % string.length
    while n < reversed_end:
        n += 1
        if reverse:
            reverse_start = reverse_start % string.length
            reverse_end = reverse_end % string.length
        elif reverse:
            reverse_end += 1
        self.string = string[n:n + reverse_end]
    return self.string.rstrip()

Now, using the example string "hello" I can reverse it:
>>> list(reverse_string("hello", reverse=True))
['h', 'e', 'l', 'o', 'r', 'h', 'e', 'l', 'o','s', 'h', 'e', 'l', 'o']()

If you want to keep the same data type, you could also do
reverse_string(self.string,reverse=True)[2:]

The only difference might be that this time it will also create an array with the reversed values of the original string, which is not a great idea as such it will lead to the strings overlapping and you won't be able to access them.

<|endoftext|>---
title: "How D

In [16]:
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.1.0+cu121
0.16.0+cu121
